In [86]:
import json
import traceback
import datetime 
from fabrictestbed_extensions.fablib.fablib import fablib

In [87]:
# Number of nodes in the cluster.
num_nodes=4

# Give a cluster name
slice_name='cassandra_2'

# Site name, pick one site from the above list of resources.
site='STAR'

# LEGACY VARIABLES
master_gpu=False


# Operation system, Linux distribution e.g. default_ubuntu_18, default_ubuntu_20, etc.
image='default_ubuntu_22'

In [88]:
node_names=[]
nic_names=[]
iface_names=[]
network_name='cluster_network'


for i in range(1,num_nodes+1):
    node_names.append("Node{0}".format(i))
    nic_names.append("Nic{0}".format(i))
    iface_names.append("iface{0}".format(i))

print(node_names)
print(nic_names)
print(iface_names)

['Node1', 'Node2', 'Node3', 'Node4']
['Nic1', 'Nic2', 'Nic3', 'Nic4']
['iface1', 'iface2', 'iface3', 'iface4']


In [89]:
try:
    slice=fablib.new_slice(name=slice_name)
    
    for i in range(num_nodes):
        if master_gpu == False and i == 0:
            node=slice.add_node(name=node_names[i],
                                site=site, 
                                cores=4,
                                ram=8,
                                disk=500,
                                image=image)
            iface_names[i]=node.add_component(model='NIC_Basic', name=node_names[i]).get_interfaces()[0]
            #node.add_component(model='NVME_P4510', name=nvme_names[i])
        
        else:
            node=slice.add_node(name=node_names[i],
                                site=site,
                                cores=4,
                                ram=8,
                                disk=500,
                                image=image)
            iface_names[i]=node.add_component(model='NIC_Basic',name=node_names[i]).get_interfaces()[0]

except Exception as e:
    print(f'Exception: {e}') 

In [90]:
try:
    net_cluster=slice.add_l2network(name=network_name, interfaces=iface_names[:])
except Exception as e:
    print(f"Exception: {e}")

In [91]:
try:
    slice.submit()
except Exception as e:
    print(f'Exception: {e}')


Retry: 10, Time: 250 sec


ID,122f89d4-be4c-4eb1-b31b-23a5c1c776ff
Name,cassandra_2
Lease Expiration (UTC),2024-12-09 02:24:09 +0000
Lease Start (UTC),2024-12-08 02:24:09 +0000
Project ID,a6504676-441c-4d76-ab04-5b13212df83d
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
083ec057-7093-489e-a9f6-cf65b204978d,Node1,4,8,500,default_ubuntu_22,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe54:cd7d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe54:cd7d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7a3dffe8-7be2-4841-b09c-c0c7643b7c23,Node2,4,8,500,default_ubuntu_22,qcow2,star-w1.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:feb3:625b,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:feb3:625b,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
012a8e8c-a8d5-4a4a-8c8b-272df96df317,Node3,4,8,500,default_ubuntu_22,qcow2,star-w1.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe9a:dd14,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe9a:dd14,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
502a6a39-42c3-4c12-9a29-d264a5acd56c,Node4,4,8,500,default_ubuntu_22,qcow2,star-w1.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:feea:d36e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:feea:d36e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
6c57f2fc-4ece-4ba9-aed7-c2ff88a9bd6b,cluster_network,L2,L2Bridge,STAR,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-Node1-p1,p1,Node1,cluster_network,100,config,,4A:CA:52:C6:7B:D2,enp7s0,enp7s0,fe80::48ca:52ff:fec6:7bd2,4,HundredGigE0/0/0/11
Node2-Node2-p1,p1,Node2,cluster_network,100,config,,02:5C:96:36:FB:07,enp7s0,enp7s0,fe80::5c:96ff:fe36:fb07,6,HundredGigE0/0/0/5
Node3-Node3-p1,p1,Node3,cluster_network,100,config,,02:76:D6:33:2A:4E,enp7s0,enp7s0,fe80::76:d6ff:fe33:2a4e,6,HundredGigE0/0/0/5
Node4-Node4-p1,p1,Node4,cluster_network,100,config,,02:85:66:88:90:43,enp7s0,enp7s0,fe80::85:66ff:fe88:9043,6,HundredGigE0/0/0/5



Time to print interfaces 265 seconds


In [92]:

from ipaddress import IPv4Address, IPv4Network

try:
    subnet=IPv4Network("192.168.1.0/24")
    available_ips=list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

In [93]:
try:
    for node in slice.get_nodes():
        node_iface=node.get_interface(network_name=network_name)
        node_IP_addr=available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_IP_addr, subnet=subnet)
        
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        _, _ = node.execute('sudo apt-get update')
        stdout, stderr = node.execute('sudo apt install net-tools')        
        stdout, stderr = node.execute(f'sudo ifconfig {node_iface.get_os_interface()} up')
        
        
except Exception as e:
    print(f"Exception: {e}")

3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 4a:ca:52:c6:7b:d2 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global enp7s0
       valid_lft forever preferred_lft forever
    inet6 fe80::48ca:52ff:fec6:7bd2/64 scope link 
       valid_lft forever preferred_lft forever
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1956 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [14.1 MB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main Translation-en [313 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 c-n-f Metadata

In [94]:
try:
    node1=slice.get_node(name='Node1')
    stdout,stderr=node1.execute(f' ping -c 3 192.168.1.4')
    print(stdout)
    print(stderr)
    
except Exception as e:
    print(f'Exception: {e}')

PING 192.168.1.4 (192.168.1.4) 56(84) bytes of data.
64 bytes from 192.168.1.4: icmp_seq=1 ttl=64 time=0.298 ms
64 bytes from 192.168.1.4: icmp_seq=2 ttl=64 time=0.061 ms
64 bytes from 192.168.1.4: icmp_seq=3 ttl=64 time=0.071 ms

--- 192.168.1.4 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2037ms
rtt min/avg/max/mdev = 0.061/0.143/0.298/0.109 ms
PING 192.168.1.4 (192.168.1.4) 56(84) bytes of data.
64 bytes from 192.168.1.4: icmp_seq=1 ttl=64 time=0.298 ms
64 bytes from 192.168.1.4: icmp_seq=2 ttl=64 time=0.061 ms
64 bytes from 192.168.1.4: icmp_seq=3 ttl=64 time=0.071 ms

--- 192.168.1.4 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2037ms
rtt min/avg/max/mdev = 0.061/0.143/0.298/0.109 ms




In [95]:

def append_line(file_path,text):
    with open(file_path,"a+") as file_des:
        file_des.seek(0)
        data=file_des.read(-1)
        if len(data)>0:
            file_des.write("\n")
        file_des.write(text)

In [96]:
import os

if os.path.isfile('/home/fabric/work/hosts') or os.path.isfile('/home/fabric/work/ips.txt') or os.path.isfile('/home/fabric/work/workers'):
    os.system("rm /home/fabric/work/hosts")
    os.system("rm /home/fabric/work/ips.txt")
    os.system("rm /home/fabric/work/workers")    
else:
    print("does not exist")

if os.path.isfile('/home/fabric/work/gpu_ips.txt'):
    os.system("rm /home/fabric/work/gpu_ips.txt")
    
#os.system("rm /home/fabric/work/ips.txt")

In [97]:
# Run this cell

# Capturing the IP addresses, this step needs to be integrated with the IP assigning stage, coded above. Or it may stay independent.
import os

i=1
local_host="127.0.0.1 localhost"
path_to_host_file="/home/fabric/work/hosts"
path_to_ip_file="/home/fabric/work/ips.txt"
path_to_worker_ip="/home/fabric/work/workers"
path_to_gpu_ips="/home/fabric/work/gpu_ips.txt"
gpu_name="NVIDIA"

append_line(path_to_host_file,local_host)

try:
    for node in slice.get_nodes():
        stdout, stderr=node.execute("hostname -I")
        IP=stdout.split(" ")[1]
        node_name="node{0}".format(i)
        vm_names="vm{0}".format(i-1)
        append_line(path_to_ip_file,IP)
        
        stdout, stderr=node.execute("hostname")
        line=IP+" "+node_name+" "+vm_names+" "+stdout
        append_line(path_to_host_file,line)
      
    
        if(i>1):
            append_line(path_to_worker_ip,vm_names)
            #append_line(path_to_worker_ip,IP)
        
        print(line)
        print(stderr)
        
        stdout_gpu, _=node.execute('lspci | grep NVIDIA')   
        if gpu_name in stdout_gpu:
            stdout_ip, _=node.execute('hostname -I')
            ip=stdout_ip.split(" ")[1]
            append_line(path_to_gpu_ips,ip)
            #print(ip)
        
        i=i+1
except Exception as e:
    print(f"Exception: {e}")
    
print(IP)

10.30.6.38 192.168.1.1 2001:400:a100:3030:f816:3eff:fe54:cd7d 
Node1
192.168.1.1 node1 vm0 Node1


10.30.6.157 192.168.1.2 2001:400:a100:3030:f816:3eff:feb3:625b 
Node2
192.168.1.2 node2 vm1 Node2


10.30.6.184 192.168.1.3 2001:400:a100:3030:f816:3eff:fe9a:dd14 
Node3
192.168.1.3 node3 vm2 Node3


10.30.6.35 192.168.1.4 2001:400:a100:3030:f816:3eff:feea:d36e 
Node4
192.168.1.4 node4 vm3 Node4


192.168.1.4


In [98]:
try:
    for node in slice.get_nodes():
        ### if you run the command twice the back up will be overwritten, a conditional block should be written
        stdout, stderr=node.execute(f'sudo cp /etc/hosts /etc/hosts_backup') 
        output_host_copy=node.upload_file(path_to_host_file,"/home/ubuntu/hosts")
        output_ip_copy=node.upload_file(path_to_ip_file,"/home/ubuntu/ips.txt")
        output_worker_copy=node.upload_file(path_to_worker_ip,"/home/ubuntu/workers")
        stdout_host_copy,stderr_host_copy=node.execute(f'sudo cp /home/ubuntu/hosts /etc/hosts')
        if os.path.isfile('/home/fabric/work/gpu_ips.txt'):
            output_gpu_copy=node.upload_file(path_to_gpu_ips,"/home/ubuntu/gpu_ips.txt")
        
        print(stderr)
        print(stderr_host_copy)
except Exception as e:
    print(f"Exception : {e}")

In [99]:
import os

output=os.system('ssh-keygen -q -t rsa -N "" -f /home/fabric/work/id_rsa > /dev/null 2>&1')

In [100]:
try:
    for node in slice.get_nodes():
        output_private=node.upload_file("/home/fabric/work/id_rsa","/home/ubuntu/.ssh/id_rsa")
        output_public=node.upload_file("/home/fabric/work/id_rsa.pub","/home/ubuntu/.ssh/id_rsa.pub")
        
        stdout, stderr=node.execute(f' cat /home/ubuntu/.ssh/id_rsa.pub >> /home/ubuntu/.ssh/authorized_keys')
        stdout, stderr=node.execute(f' chmod 600 /home/ubuntu/.ssh/id_rsa*')
        
        print(output_private)
        print(output_public)
        print(stderr)
        #print(stdout)
        
except Exception as e:
    print(f"Exception: {e}")

-rw-rw-r--   1 1000     1000         2622 08 Dec 02:31 ?
-rw-rw-r--   1 1000     1000          587 08 Dec 02:31 ?

-rw-rw-r--   1 1000     1000         2622 08 Dec 02:32 ?
-rw-rw-r--   1 1000     1000          587 08 Dec 02:32 ?

-rw-rw-r--   1 1000     1000         2622 08 Dec 02:32 ?
-rw-rw-r--   1 1000     1000          587 08 Dec 02:32 ?

-rw-rw-r--   1 1000     1000         2622 08 Dec 02:32 ?
-rw-rw-r--   1 1000     1000          587 08 Dec 02:32 ?

